In [1]:
%config IPCompleter.greedy=True

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [126]:
s2_data = pd.read_csv("scenario2_tests.csv")
s2_data

,n,slots
0,100,3
1,100,6
2,100,1
3,100,6
4,100,1
...,...,...
99995,10000,1
99996,10000,5
99997,10000,2
99998,10000,1


In [127]:
s3_data = pd.read_csv("scenario3_tests.csv")
s3_data

,u,n,slots,rounds
0,100,2,1,1
1,100,2,1,1
2,100,2,1,1
3,100,2,1,1
4,100,2,1,1
...,...,...,...,...
146995,4900,4900,12,1
146996,4900,4900,11,1
146997,4900,4900,12,1
146998,4900,4900,11,3


# Exercise 2

In [167]:
s3_summary = s3_data[27000:30000].groupby(['u', 'n', 'rounds'])['slots'].unique()
s3_summary

u     n     rounds
1000  2     1         [1, 5, 3, 2, 4, 6, 8, 7, 9]
            2                  [1, 2, 5, 3, 4, 8]
            3                     [1, 2, 4, 3, 5]
            4                              [1, 2]
            5                                 [2]
      500   1                    [7, 10, 8, 9, 6]
            2                    [7, 10, 9, 8, 6]
            3                       [9, 10, 8, 7]
            4                           [7, 9, 8]
            5                           [9, 8, 7]
            6                                 [9]
      1000  1                       [9, 10, 8, 7]
            2                       [10, 9, 8, 7]
            3                          [9, 10, 8]
            4                          [9, 10, 8]
            5                          [10, 9, 8]
            6                             [9, 10]
            7                             [10, 9]
            8                             [10, 9]
Name: slots, dtype: object